## Scraping [1/2]

In [46]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.spin-off.fr/calendrier_des_series.html'
response = requests.get(url)
content = response.content
page = BeautifulSoup(content, 'html')


currentMonth = page.find_all('td', class_=['floatleftmobile td_jour', 'floatleftmobile td_jour td_jourcourant'])
for day in currentMonth:
  dateElement = day.find('div', class_=['div_jour', 'div_jourcourant fond_degrade_v4'])
  if dateElement:
    date = dateElement.get('id').split('_')[1]

    # if date == '03-11-2023': #test different date
    print(date)

    episodes = day.find_all('span', class_=['calendrier_episodes'])
    for episodeElement in episodes:
      episodeNameElements = episodeElement.findAll('a')
      seasonAndEpisode = episodeNameElements[1].text.split('.')
      channelElement = episodeElement.find_previous_sibling('img')
      contryElements = channelElement.find_previous_sibling('img')
      episodeObject = {
        'name': episodeNameElements[0].text,
        'date': date,
        'episode': seasonAndEpisode[1],
        'season': seasonAndEpisode[0],
        'country': contryElements.get('alt'),
        'channel': channelElement.get('alt'),
        'url': episodeNameElements[0].get('href')
      }
      print(episodeObject)
    print('   ')

01-11-2023
{'name': '4 Estrellas', 'date': '01-11-2023', 'episode': '110', 'season': '1', 'country': 'Espagne', 'channel': 'TVE', 'url': 'serie-15345-4-Estrellas.html'}
{'name': 'Black Cake', 'date': '01-11-2023', 'episode': '01', 'season': '1', 'country': 'Etats-Unis', 'channel': 'Hulu', 'url': 'serie-15614-Black-Cake.html'}
{'name': 'Black Cake', 'date': '01-11-2023', 'episode': '02', 'season': '1', 'country': 'Etats-Unis', 'channel': 'Hulu', 'url': 'serie-15614-Black-Cake.html'}
{'name': 'Black Cake', 'date': '01-11-2023', 'episode': '03', 'season': '1', 'country': 'Etats-Unis', 'channel': 'Hulu', 'url': 'serie-15614-Black-Cake.html'}
{'name': 'Blanca', 'date': '01-11-2023', 'episode': '05', 'season': '2', 'country': 'Italie', 'channel': 'Rai 1', 'url': 'serie-14708-Blanca.html'}
{'name': 'Children Ruin Everything', 'date': '01-11-2023', 'episode': '06', 'season': '3', 'country': 'Canada', 'channel': 'CTV', 'url': 'serie-13789-Children-Ruin-Everything.html'}
{'name': 'Chucky', 'date